# Spark Structured Streaming with Kafka Integration

## Additional Dependency

Submit via shell:


```shell
spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0
```

## SparkSession

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName(
    "Structured Streaming Kafka Integration"
).getOrCreate()


## Reading JSON from Kafka

In [ ]:
boom = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka.nadzieja.test:9092") # broker nodes
    .option("subscribe", "dictionary") # topic to subscribe
    .load()
)

boom1 = boom.selectExpr("from_json(cast(value as string), 'nama STRING, alasan STRING') as message")

## Writing to Kafka

In [ ]:
boom2 = boom1.select("message.nama")

boomWriter = (
    boom2.filter("nama is not null")
    .selectExpr(
        # to write data to Kafka, dataframe needs to have key column and value column
        "cast(null as string) as key",
        "to_json(struct(nama)) as value",
    )
    .writeStream.format("kafka")
    .option("topic", "people")  # topic to write
    .option("kafka.bootstrap.servers", "kafka.nadzieja.test:9092")  # broker
)

boomWriter.option("checkpointLocation", "checkpoint").start()
